<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN/CNN_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1-import library
 
  إستيراد المكتبات

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('stopwords')
from nltk.stem.isri import ISRIStemmer
import re
import string

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


--------

2- import Datasets

استيراد ملفات البيانات


In [2]:
# dataset = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/3.xlsx")
dataset = pd.read_excel("drive/MyDrive/mining/4.xlsx")

dataset.info()

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

------
3- shuffle  
بعثرة البيانات

In [ ]:
dataset=shuffle(dataset)
dataset

,text,targe
29519,تعهد سعد الدين العثماني رئيس الحكومه الجديد بم...,3
42321,عبر عد من الشباب الحاملين لمشاريع بمدينه طنجه ...,2
10209,منعت السلطات الاسبانيه مدير مكتب الجزيره باسبا...,2
30207,وصف الميلودي مخاريق امين عام الاتحاد المغربي ل...,3
9128,عبدالاله بوسحابه اخبارنا المغربيه بعد الفضيحه ...,2
...,...,...
12824,اكد حزب العداله والتنميه الاسلامي الذي يقود ال...,3
41515,درك الحربي اكتشف عناصر جرميه في تلقي الجنود لر...,1
36695,شذي يتابع ملاين العرب علي مواقع التواصل الاجتم...,0
48325,اعتبرت الملكه المغربيه مساء الثلاثاء ان الجوء ...,3


------
4- clean
تنظيف البيانات

In [ ]:
print(np.sum(dataset.isnull().any(axis=1)))
dataset= dataset.dropna()
print(np.sum(dataset.isnull().any(axis=1)))

#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
           
remove_hashtag(dataset)
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc
    
stemmer = ISRIStemmer()
dataset["text"] = dataset['text'].apply(lambda x: processDocument(x, stemmer))
print(dataset)

reviews = dataset['text'].values
t = Tokenizer(oov_token='<UNK>')
# fit the tokenizer on the documents
t.fit_on_texts(reviews)
t.word_index['<PAD>'] = 0

VOCAB_SIZE = len(t.word_index)

0
0
                                                    text  targe
29519  تعهد سعد الدين العثماني رئيس الحكومه الجديد بم...      3
42321  عبر عد من الشباب الحاملين لمشاريع بمدينه طنجه ...      2
10209  منعت السلطات الاسبانيه مدير مكتب الجزيره باسبا...      2
30207  وصف الميلودي مخاريق امين عام الاتحاد المغربي ل...      3
9128   عبدالاله بوسحابه اخبارنا المغربيه بعد الفضيحه ...      2
...                                                  ...    ...
12824  اكد حزب العداله والتنميه الاسلامي الذي يقود ال...      3
41515  درك الحربي اكتشف عناصر جرميه في تلقي الجنود لر...      1
36695  شذي يتابع ملاين العرب علي مواقع التواصل الاجتم...      0
48325  اعتبرت الملكه المغربيه مساء الثلاثاء ان الجوء ...      3
25768  اقتنت الشركه الماليه الدوليه فرع البند الدولي ...      2

[52133 rows x 2 columns]


In [ ]:
def Tokenizer_UNK(train_x,test_reviews,train_y, test_y):
  norm_train_reviews = (train_x)
  norm_test_reviews = (test_reviews)

  le = LabelEncoder()
  y_train = le.fit_transform(train_y)
  y_test = le.transform(test_y)

  from keras.utils.np_utils import to_categorical
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  train_sequences = t.texts_to_sequences(norm_train_reviews)
  test_sequences = t.texts_to_sequences(norm_test_reviews)
  print("Vocabulary size={}".format(len(t.word_index)))
  print("Number of Documents={}".format(t.document_count))

  MAX_SEQUENCE_LENGTH = 1000
  # pad dataset to a maximum review length in words
  X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
  X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
  # X_train.shape, X_test.shape
  return X_train, X_test ,y_train,y_test

In [ ]:
def set_model():
  VOCAB_SIZE = len(t.word_index)
  MAX_SEQUENCE_LENGTH = 1000
  EMBED_SIZE = 300
  
  # create the model
  global model
  model = Sequential()
  model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
  model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))

  # model.add(Dense(1, activation='sigmoid'))
  model.add(Dense(5, activation='sigmoid'))
  # model.add(Flatten())
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  return model

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold

# build train and test datasets
reviews = dataset['text'].values
sentiments = dataset['targe'].values
#####
strtfdKFold = StratifiedKFold(n_splits=4)#5
kfold = strtfdKFold.split(reviews, sentiments)
result_CNNClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    # if(os.path.exists("drive/MyDrive/mining/CNN.net")):
    #   print("load net")
    #   model = keras.models.load_model('drive/MyDrive/mining/CNN.net') 

    X_train_, X_test_ = reviews[train_index], reviews[test_index]
    y_train_, y_test_ = sentiments[train_index], sentiments[test_index]

    X_train, X_test ,y_train,y_test=Tokenizer_UNK(X_train_,X_test_,y_train_,y_test_)
    # set_model()

    # Fit the model
    model=set_model()
    EPOCHS=2  
    BATCH_SIZE=128
    model.fit(X_train, y_train, validation_split=0.1, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
    y_pred=(model.predict(X_test))
    y_classes_pre = [np.argmax(y, axis=None, out=None) for y in y_pred]
    y_classes_test = [np.argmax(y, axis=None, out=None) for y in y_test]
    result_CNNClassifier.append(calculate_results(y_classes_test, y_classes_pre))
    # model.save('drive/MyDrive/mining/CNN.net')
    # print("save net")

result_CNNClassifier

In [ ]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_CNNClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_CNNClassifier)
f1=f1/len(result_CNNClassifier)
precision=precision/len(result_CNNClassifier)
recall=recall/len(result_CNNClassifier)
# (result_DecisionTreeClassifier)
results_CNNClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_CNNClassifier

[{'accuracy': 94.78257724360446,
  'f1': 0.9477387119920051,
  'precision': 0.9479345864832764,
  'recall': 0.9478257724360446}]

In [ ]:
print('DecisionTreeClassifier results in dataset 1:')
results_CNNClassifier

DecisionTreeClassifier results in dataset 1:


[{'accuracy': 95.19608342701729,
  'f1': 0.9520984811390116,
  'precision': 0.952799253715478,
  'recall': 0.9519608342701729}]

In [ ]:
print('DecisionTreeClassifier results in dataset 2:')
results_CNNClassifier

DecisionTreeClassifier results in dataset 2:


[{'accuracy': 95.57787784320516,
  'f1': 0.9558535748300416,
  'precision': 0.9565019386666695,
  'recall': 0.9557787784320517}]

In [ ]:
print('DecisionTreeClassifier results in dataset 3:')
results_CNNClassifier

DecisionTreeClassifier results in dataset 3:


[{'accuracy': 94.5945934848568,
  'f1': 0.9460234167421608,
  'precision': 0.9465212822533673,
  'recall': 0.9459459348485679}]

In [ ]:
print('DecisionTreeClassifier results in dataset 4:')
results_CNNClassifier

DecisionTreeClassifier results in dataset 4:


[{'accuracy': 94.78257724360446,
  'f1': 0.9477387119920051,
  'precision': 0.9479345864832764,
  'recall': 0.9478257724360446}]

In [ ]:
Final_Result=[
  {'accuracy': 95.19608342701729,
  'f1': 0.9520984811390116,
  'precision': 0.952799253715478,
  'recall': 0.9519608342701729}
  ,
  {'accuracy': 95.57787784320516,
  'f1': 0.9558535748300416,
  'precision': 0.9565019386666695,
  'recall': 0.9557787784320517}
  ,
  {'accuracy': 94.5945934848568,
  'f1': 0.9460234167421608,
  'precision': 0.9465212822533673,
  'recall': 0.9459459348485679}
  ,
 {'accuracy': 94.78257724360446,
  'f1': 0.9477387119920051,
  'precision': 0.9479345864832764,
  'recall': 0.9478257724360446}
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]

print('Final_Result:')
print(Final_Result)

Final_Result:
[{'accuracy': 95.03778299967092, 'f1': 0.9504285461758049, 'precision': 0.9509392652796977, 'recall': 0.9503778299967094}]


---------------
